In [1]:
import wntr
import wntr.network.controls as controls
import re

In [2]:
inp_file_path = "../Demand_patterns/ctown_map_with_controls.inp"
wn = wntr.network.WaterNetworkModel(inp_file_path)
cpa_file_path = "../Demand_patterns/ctown.cpa"

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "../Demand_patterns/ctown_map_with_controls.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [4]:
def get_control_rule_with_tag(a_tag):
    for control in control_list:
        if a_tag in control['actuator_tag']:
            return control

In [5]:
def create_control_list():
    control_list = []
    for control in wn.controls():
        control_dict = {}
        control_dict['tank_tag'] = control[1].condition.name.split(":")[0]
        control_dict['operator'] = control[1].condition.name.split("level")[1][0:2]
        control_dict['value'] = control[1].condition.name.split("=")[1]
        control_dict['actuator_tag'] = str(control[1].actions()[0].target()[0])
        control_dict['actuator_value'] = str(control[1].actions()[0]).split("to")[1].strip()    
        control_list.append(control_dict)                     
    return control_list

In [6]:
def create_plc_list():
    section_exp = re.compile(r'\[(.*)\]')
    plc_exp = re.compile(r'^PLC')
    plcs = []
    nodes_section = False
    with open(cpa_file_path, 'r') as file_object:
            for line in file_object:
                if section_exp.match(line) and line == '[CYBERNODES]\n':
                    nodes_section=True
                    continue
                if section_exp.match(line) and line != '[CYBERNODES]\n':                
                    nodes_section=False
                if nodes_section and line[0] != ";":                
                    nodes=line.split(",")
                    if plc_exp.match(nodes[0].strip()):
                        plc_dict = {}
                        plc_dict['PLC'] = nodes[0].strip()                                                
                        plc_dict['Sensors'] = nodes[1].strip().split(" ")
                        plc_dict['Actuators'] = nodes[2].strip().split(" ")
                        plc_dict['Controls'] = []
                        plcs.append(plc_dict)                                                  
    return plcs

In [7]:
def set_control_list_in_plc(a_plc_dict, a_control_list):
    plc_controls = []
    for actuator in a_plc_dict['Actuators']:
        for control in a_control_list:
            control['actuator_tag']
            if actuator == control['actuator_tag']:
                plc_controls.append(control)                
    a_plc_dict['Controls'] = plc_controls 
    return a_plc_dict 

In [8]:
def set_dependencies_list_in_plc(a_plc_list, a_control_list):
    for plc in a_plc_list:
        dependencies_list = []        
        tag_set = set()        
        for control in plc['Controls']:
            tag_set.add(control['tank_tag'])
        if tag_set:            
            for tag in tag_set:
                for target_plc in a_plc_list:                    
                    for sensor in target_plc['Sensors']:                        
                        if tag == sensor:
                            dependency_dict = {}
                            dependency_dict['tag'] = tag
                            dependency_dict['PLC'] = target_plc['PLC']
                            dependencies_list.append(dependency_dict)        
        plc['Dependencies'] = dependencies_list                

In [9]:
def get_sensor_initial_value(sensor_tag):
    return wn.get_node(sensor_tag).init_level

In [10]:
def get_actuator_initial_value(actuator_tag):
    if type(wn.get_link(actuator_tag).status) is int:
        return wn.get_link(actuator_tag).status
    else:
        return wn.get_link(actuator_tag).status.value

In [11]:
def get_link_list_by_type(a_list, a_type):
    result = []
    for link in a_list:
        if wn.get_link(link).link_type == a_type:
            result.append(str(link))
    return result

In [12]:
def get_node_list_by_type(a_list, a_type):
    result = []
    for node in a_list:
        if wn.get_node(node).node_type == a_type:
            result.append(str(node))
    return result

In [13]:
control_list = create_control_list()
plc_list = create_plc_list()

for i in range(len(plc_list)):
    plc_list[i] = set_control_list_in_plc(plc_list[i], control_list)

set_dependencies_list_in_plc(plc_list, control_list)
    
for plc in plc_list:

    print(plc)
    print("\n")

{'PLC': 'PLC1', 'Sensors': [''], 'Actuators': ['PU1', 'PU2', 'PU3'], 'Controls': [{'tank_tag': 'T1', 'operator': '<=', 'value': '4.0', 'actuator_tag': 'PU1', 'actuator_value': 'Open'}, {'tank_tag': 'T1', 'operator': '>=', 'value': '6.3', 'actuator_tag': 'PU1', 'actuator_value': 'Closed'}, {'tank_tag': 'T1', 'operator': '<=', 'value': '1.0', 'actuator_tag': 'PU2', 'actuator_value': 'Open'}, {'tank_tag': 'T1', 'operator': '>=', 'value': '4.5', 'actuator_tag': 'PU2', 'actuator_value': 'Closed'}], 'Dependencies': [{'tag': 'T1', 'PLC': 'PLC2'}]}


{'PLC': 'PLC2', 'Sensors': ['T1'], 'Actuators': [''], 'Controls': [], 'Dependencies': []}


{'PLC': 'PLC3', 'Sensors': ['T2'], 'Actuators': ['PU4', 'PU5', 'PU6', 'PU7', 'V2'], 'Controls': [{'tank_tag': 'T3', 'operator': '<=', 'value': '3.0', 'actuator_tag': 'PU4', 'actuator_value': 'Open'}, {'tank_tag': 'T3', 'operator': '>=', 'value': '5.3', 'actuator_tag': 'PU4', 'actuator_value': 'Closed'}, {'tank_tag': 'T3', 'operator': '<=', 'value': '1.0', '

In [14]:
# /////////////////////////////////// This section will build a utils.py file with the dictionary //////////// #############

######## Network Information

# topology name
topology_name = "ctown"

#Constants
plc_netmask="'/24'"
enip_listen_plc_addr="'192.168.1.1'"
scada_ip_addr="'192.168.1.2'"

### Creating the dictionary with the IP addresses of the PLCs
plc_index = 1
ip_string = ""

# PLC DATA TODO section
plc_data_string = ""

# PLC Tags
plc_tags = []

# PLC Servers
plc_servers = ""

# PLC protocols 
plc_protocols = ""

# SCADA information
scada_tags = "SCADA_TAGS = ("
scada_server = "SCADA_SERVER = {\n    'address': SCADA_IP_ADDR,\n    'tags': SCADA_TAGS\n}\n"
scada_protocol = "SCADA_PROTOCOL = {\n    'name': 'enip',\n    'mode': 1,\n    'server': SCADA_SERVER\n}\n"


####### For the DB tags, is better to refer to WNTR as epanet.INP and cpa files may not provide the full list of pumps and valves
# Creating the tags strings. We need to differentiate between sensor and actuator tags, because we get their initial values/status in different ways from WaterNetworkModel

db_sensor_list = []
db_actuator_list = []
db_tags = []
a_link_list = list(wn.link_name_list)
a_node_list = list(wn.node_name_list)
db_actuator_list.extend(get_link_list_by_type(a_link_list, 'Valve'))
db_actuator_list.extend(get_link_list_by_type(a_link_list, 'Pump'))
db_sensor_list.extend(get_node_list_by_type(a_node_list, 'Tank'))

db_tags.extend(db_actuator_list)
db_tags.extend(db_sensor_list)

tags_strings = []

for tag in db_tags:
    tag_dict = {}
    tag_dict['name'] = tag
    tag_dict['string'] = "('" + tag + "', 1)"    
    tags_strings.append(tag_dict)
    scada_tags+="\n    ('" + tag + "', 1, 'REAL'),"

scada_tags = scada_tags + "\n)\n"

# Control fag to sync actuators and physical process
tag_dict = {}
tag_dict['name'] = 'CONTROL'
tag_dict['string'] = "('CONTROL', 1)"    
tags_strings.append(tag_dict)
############## PLC ENIP address configuration

for plc in plc_list: 
    ip_string+="    '" + plc['PLC'].lower() + "':'10.0." + str(plc_index) + ".1',\n"

    plc_data_string += "\nPLC" + str(plc_index) + "_DATA = {\n    'TODO': 'TODO',\n}\n"
    a_plc_tag_string = "PLC" + str(plc_index) + "_TAGS = ("
    tag_string = ""
    for tag in plc['Sensors']:
        if tag != "":
            tag_string+="\n    ('" + tag + "', 1, 'REAL'),"

    for tag in plc['Actuators']:
        if tag != "":
            tag_string+="\n    ('" + tag + "', 1, 'REAL'),"

    for dependency in plc['Dependencies']:
        if dependency:
            tag_string+="\n    ('" + dependency['tag'] + "', 1, 'REAL'),"
    
    tag_string += "\n    ('CONTROL', 1, 'REAL'),"
    
    a_plc_tag_string += tag_string + "\n)"
    print(a_plc_tag_string)
    plc_tags.append(a_plc_tag_string)
    
    plc_servers += "PLC" + str(plc_index) + "_SERVER = {\n    'address': ENIP_LISTEN_PLC_ADDR,\n    'tags': PLC" + str(plc_index) + "_TAGS\n}\n"
    plc_protocols += "PLC" + str(plc_index) + "_PROTOCOL = {\n    'name': 'enip',\n    'mode': 1,\n    'server': PLC" + str(plc_index) + "_SERVER\n}\n"
    plc_index+=1

    
plc_data_string+= "SCADA_DATA = {\n    'TODO': 'TODO',\n}\n"
    
ctown_ips_prefix = "CTOWN_IPS = {\n"
ctown_ips = ctown_ips_prefix + ip_string + "}"


################### DB creation strings

path_string = "PATH = 'ctown_db.sqlite'"
name_string = "NAME = '" + topology_name + "'"

state_string = "STATE = {\n    'name': NAME,\n    'path': PATH\n}"

comas = '"'*3 
schema_string = 'SCHEMA = ' + comas +'\
\nCREATE TABLE ' + topology_name + ' (\
\n    name              TEXT NOT NULL,\
\n    pid               INTEGER NOT NULL,\
\n    value             TEXT,\
\n    PRIMARY KEY (name, pid)\
\n);\n'+ comas


db_tag_string = "SCHEMA_INIT = " + comas + "\n"
for tag in db_sensor_list:
    db_tag_string += "    INSERT INTO " + topology_name + " VALUES ('" + tag +"', 1, '" + str(get_sensor_initial_value(tag)) + "');\n"

for tag in db_actuator_list:
    db_tag_string += "    INSERT INTO " + topology_name + " VALUES ('" + tag +"', 1, '" + str(get_actuator_initial_value(tag)) + "');\n"    
    
db_tag_string += "    INSERT INTO " + topology_name +" VALUES ('ATT_1', 1, '0');\n"
db_tag_string += "    INSERT INTO " + topology_name +" VALUES ('ATT_2', 1, '0');\n"
db_tag_string += "    INSERT INTO " + topology_name +" VALUES ('CONTROL', 1, '0');\n"
db_tag_string += comas

PLC1_TAGS = (
    ('PU1', 1, 'REAL'),
    ('PU2', 1, 'REAL'),
    ('PU3', 1, 'REAL'),
    ('T1', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC2_TAGS = (
    ('T1', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC3_TAGS = (
    ('T2', 1, 'REAL'),
    ('PU4', 1, 'REAL'),
    ('PU5', 1, 'REAL'),
    ('PU6', 1, 'REAL'),
    ('PU7', 1, 'REAL'),
    ('V2', 1, 'REAL'),
    ('T4', 1, 'REAL'),
    ('T3', 1, 'REAL'),
    ('T2', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC4_TAGS = (
    ('T3', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC5_TAGS = (
    ('PU8', 1, 'REAL'),
    ('PU9', 1, 'REAL'),
    ('PU10', 1, 'REAL'),
    ('PU11', 1, 'REAL'),
    ('T7', 1, 'REAL'),
    ('T5', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC6_TAGS = (
    ('T4', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC7_TAGS = (
    ('T5', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC8_TAGS = (
    ('T6', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)
PLC9_TAGS = (
    ('T7', 1, 'REAL'),
    ('CONTROL', 1, 'REAL'),
)


In [15]:
print(db_actuator_list)
print(db_sensor_list)

['v1', 'V45', 'V47', 'V2', 'PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11']
['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']


In [16]:
# Erase the file contents
utils_file = open("utils.py", "w")
utils_file.write("")
utils_file.close()

utils_file = open("utils.py", "a")
for tag in tags_strings:
    utils_file.write(tag['name'] + " = " + tag['string'] + "\n" )  

utils_file.write("\nplc_netmask = " + plc_netmask + "\n")
utils_file.write("ENIP_LISTEN_PLC_ADDR = " + enip_listen_plc_addr + "\n")
utils_file.write("SCADA_IP_ADDR = " + scada_ip_addr + "\n")

utils_file.write("\n" + ctown_ips + "\n")
utils_file.write(plc_data_string + "\n")

for plc in plc_tags:
    utils_file.write("\n" + plc + "\n")

utils_file.write("\n"+scada_tags)

# this is only temporal. With the revamp of the attacks, this will be done differently
utils_file.write("\nflag_attack_communication_plc1_plc2_replay_empty = 0\n")
utils_file.write("flag_attack_plc1 = 0\n")
utils_file.write("flag_attack_communication_plc1_plc2 = 0\n")
utils_file.write("ATT_1 = ('ATT_1', 1)\n")
utils_file.write("ATT_2 = ('ATT_2', 1)\n")
utils_file.write("\n" + plc_servers)
utils_file.write("\n" + scada_server)
utils_file.write("\n" + plc_protocols)
utils_file.write("\n"+ scada_protocol)
utils_file.write("\n" + path_string)
utils_file.write("\n" + name_string + "\n")
utils_file.write("\n" + state_string+ "\n")
utils_file.write("\n" + schema_string+ "\n")
utils_file.write("\n" + db_tag_string)

utils_file.close()

In [17]:
mask_plc1 = 1
mask_plc3 = 2
mask_plc4 = 4
mask_full = 7
control = 0

if control == 0 or not (mask_plc1 & control):
    print("PLC1 should control")
    control += mask_plc1
if control == 0 or not (mask_plc3 & control):
    print("PLC3 should control")
    control += mask_plc3
if control == 0 or not (mask_plc4 & control):
    print("PLC4 should control")
    control += mask_plc4    
if control and mask_full:
    print("Physical process should run")
    control = 0
if control == 0:     
    print("No control")
    
print("\nNew iteration\n")

if control == 0 or not (mask_plc1 & control):
    print("PLC1 should control")
    control += mask_plc1
if control == 0 or not (mask_plc3 & control):
    print("PLC3 should control")
    control += mask_plc3
if control == 0 or not (mask_plc4 & control):
    print("PLC4 should control")
    control += mask_plc4    
if control and mask_full:
    print("Physical process should run")
    control = 0
if control == 0:     
    print("No control")

PLC1 should control
PLC3 should control
PLC4 should control
Physical process should run
No control

New iteration

PLC1 should control
PLC3 should control
PLC4 should control
Physical process should run
No control
